In [1]:
import os, re, shutil, javalang
import pandas as pd

metrics: 

In [2]:
mt_field = pd.read_csv(r"eclipse_field.csv")
mt_method = pd.read_csv(r"eclipse_method.csv")
mt_file = pd.read_csv(r"eclipse_count_interface_class.csv")

In [3]:
mt_field.head()

,file,NOF_avg,NOF_max,NOF_total,NSF_avg,NSF_max,NSF_total
0,$packageName$.$actionClass$.java,0.0,0,0,0.0,0,0
1,$packageName$.$contributorClassName$.java,2.0,2,2,0.0,0,0
2,$packageName$.$editorClass$.java,1.0,1,1,0.0,0,0
3,$packageName$.$editorClassName$.java,3.0,3,3,0.0,0,0
4,$packageName$.$wizardClassName$.java,2.0,2,2,0.0,0,0


In [4]:
mt_method.head()

,file,NOM_avg,NOM_max,NOM_total,NSM_avg,NSM_max,NSM_total
0,$packageName$.$actionClass$.java,3.0,3,3,0.0,0,0
1,$packageName$.$contributorClassName$.java,6.0,6,6,0.0,0,0
2,$packageName$.$editorClass$.java,1.0,1,1,0.0,0,0
3,$packageName$.$editorClassName$.java,16.0,16,16,0.0,0,0
4,$packageName$.$wizardClassName$.java,8.0,8,8,0.0,0,0


In [5]:
mt_file.head()

,file,num_of_anony,num_of_interface,num_of_class,line_of_code
0,$packageName$.$actionClass$.java,??,0,1,42
1,$packageName$.$contributorClassName$.java,??,0,1,103
2,$packageName$.$editorClass$.java,??,0,1,20
3,$packageName$.$editorClassName$.java,??,0,1,239
4,$packageName$.$wizardClassName$.java,??,0,1,150


In [6]:
_ = pd.merge(mt_field,mt_method,on=['file'],how= "inner")
mt = pd.merge(_,mt_file,on=['file'],how= "inner")
mt.head()

,file,NOF_avg,NOF_max,NOF_total,NSF_avg,NSF_max,NSF_total,NOM_avg,NOM_max,NOM_total,NSM_avg,NSM_max,NSM_total,num_of_anony,num_of_interface,num_of_class,line_of_code
0,$packageName$.$actionClass$.java,0.0,0,0,0.0,0,0,3.0,3,3,0.0,0,0,??,0,1,42
1,$packageName$.$contributorClassName$.java,2.0,2,2,0.0,0,0,6.0,6,6,0.0,0,0,??,0,1,103
2,$packageName$.$editorClass$.java,1.0,1,1,0.0,0,0,1.0,1,1,0.0,0,0,??,0,1,20
3,$packageName$.$editorClassName$.java,3.0,3,3,0.0,0,0,16.0,16,16,0.0,0,0,??,0,1,239
4,$packageName$.$wizardClassName$.java,2.0,2,2,0.0,0,0,8.0,8,8,0.0,0,0,??,0,1,150


post and pre bug

In [7]:
post_bug = pd.read_csv(r"eclipse_post_bug.csv")
pre_bug = pd.read_csv(r"eclipse_pre_bug.csv")

In [8]:
post_bug.head()

,file,commitNo,date,bugID,releaseNo,Date,Changed,Changed-Date
0,eclipse\org.eclipse.equinox.ds\src\org\eclipse...,1.4,2006-05-24,143576,R3_2_2,2007-03-01,2007-06-07,98 days 00:00:00.000000000
1,eclipse\e4\org.eclipse.e4.resources\bundles\or...,1.45.2.1,2008-08-22,232426,R3_4_1,2008-09-24,2008-10-16,22 days 00:00:00.000000000
2,eclipse\e4\org.eclipse.e4.resources\bundles\or...,1.15.8.1,2008-08-22,232426,R3_4_1,2008-09-24,2008-10-16,22 days 00:00:00.000000000
3,eclipse\e4\org.eclipse.e4.resources\tests\org....,1.17.2.1,2008-08-22,232426,R3_4_1,2008-09-24,2008-10-16,22 days 00:00:00.000000000
4,eclipse\e4\org.eclipse.e4.resources\tests\org....,1.1.2.2,2008-08-22,232426,R3_4_1,2008-09-24,2008-10-16,22 days 00:00:00.000000000
